### Motivations
Sometimes we may want to operate on all the tokens in a given place node or nodes.

Here we have a simple example of using a transition function to pair up tokens from two different place nodes.


### This Example
This is a simple example where string tokens are matched against integers corresponding to the number of characters in the string.

There may not necessarily be a unique set of matches and not every token may match up.


### Petri Net Mechanics
The idea here is to use type annotations to indicate that rather than being passed a single token from a place node,
all the tokens should be passed in.

In [ ]:
from typing import Optional
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization

from petritype.core.executable_graph_components import (
    ListPlaceNode, FunctionTransitionNode, ArgumentEdgeToTransition, ReturnedEdgeFromTransition,
    ExecutableGraphOperations
)
from petritype.core.rustworkx_graph import RustworkxGraph
from petritype.plotting.simple_graphviz import SimpleGraphvizVisualization
from IPython.display import display, clear_output

In [ ]:
def match_one_string_to_one_length(strings: list[str], lengths: list[int]) -> Optional[tuple[str, int]]:
    for s in strings:
        for l in lengths:
            if len(s) == l:
                return (s, l)
    return None

In [ ]:
executable_graph_nodes_and_edges = [
    ListPlaceNode("Some Strings", str, ["a", "ab", "abc", "abcd"]),
    ListPlaceNode("Some Lengths", int, [2, 3, 4, 5, 99]),
    
    ArgumentEdgeToTransition("Some Strings", "Match Lengths", "strings"),
    ArgumentEdgeToTransition("Some Lengths", "Match Lengths", "lengths"),
    FunctionTransitionNode("Match Lengths", match_one_string_to_one_length),
    ReturnedEdgeFromTransition("Match Lengths", "Matched Pair"),

    ListPlaceNode("Matched Pair", tuple[str, int], []),
]

In [ ]:
executable_graph = ExecutableGraphOperations.construct_graph(executable_graph_nodes_and_edges)
executable_pydigraph = RustworkxGraph.from_executable_graph(executable_graph)
SimpleGraphvizVisualization.graph(executable_pydigraph)

In [ ]:
async for step, diagram, transitions_fired in SimpleGraphvizVisualization.animate_execution_generator(
    executable_graph=executable_graph,
    executable_pydigraph=executable_pydigraph,
):
    clear_output(wait=True)
    print(f"Step {step}")
    display(diagram)
    print(f"Transitions fired: {transitions_fired}")
    if not transitions_fired:
        print("No more transitions to fire. Execution complete.")
        break
    time.sleep(1.0)
    plt.close()

In [ ]:
# Oops the tokens were lost!